<a href="https://colab.research.google.com/github/Giffy/fast.ai/blob/master/Machine%20Learning/lesson3_grocery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important: This notebook will only work with fastai-0.7.x. Do not try to run any fastai-1.x code from this path in the repository because it will load fastai-0.7.x**

# Intro to Random Forests - Favorita Grocery Sales Forecasting 

Notebook based in kaggle competition [Can you accurately predict sales for a large grocery chain?](https://www.kaggle.com/c/favorita-grocery-sales-forecasting)

Corporación Favorita has challenged the Kaggle community to build a model that more accurately forecasts product sales.

They’re excited to see how machine learning could better ensure they please customers by having just enough of the right products at the right time.

In this competition, you will be predicting the unit sales for thousands of items sold at different Favorita stores located in Ecuador. The training data includes dates, store and item information, whether that item was being promoted, as well as the unit sales. Additional files include supplementary information that may be useful in building your models.

##File Descriptions and Data Field Information

**train.csv**

* Training data, which includes the target unit_sales by date, store_nbr, and item_nbr and a unique id to label rows.
* The target unit_sales can be integer (e.g., a bag of chips) or float (e.g., 1.5 kg of cheese).
* Negative values of unit_sales represent returns of that particular item.
* The onpromotion column tells whether that item_nbr was on promotion for a specified date and store_nbr.
* Approximately 16% of the onpromotion values in this file are NaN.
* **NOTE:** The training data does not include rows for items that had zero unit_sales for a store/date combination. There is no information as to whether or not the item was in stock for the store on the date, and teams will need to decide the best way to handle that situation. Also, there are a small number of items seen in the training data that aren't seen in the test data.


**test.csv**

* Test data, with the date, store_nbr, item_nbr combinations that are to be predicted, along with the onpromotion information.
* **NOTE:** The test data has a small number of items that are not contained in the training data. Part of the exercise will be to predict a new item sales based on similar products..
* The public / private leaderboard split is based on time. All items in the public split are also included in the private split.


# Google Colab setup
Installs fast.ai 0.7.0 and the required libraries to run the notebook.

Also downloads the required datasets.

Train.csv dataset has been reduced in order to store the data sample in Github. 

Original data file can be downloaded in [kaggle](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data) (size of compressed file:  474Mb) 

In [0]:
print (" Installing FastAI libraries ... (takes 2 min)")
!pip install fastai==0.7.0 > /dev/null
print ("\n Clonning FastAI repository locally ...")
!git clone https://github.com/fastai/fastai.git fastai_ml
!ln -s fastai_ml/courses/ml1/fastai/ fastai

 Installing FastAI libraries ... (takes 2 min)
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
mizani 0.5.3 has requirement pandas>=0.23.4, but you'll have pandas 0.22.0 which is incompatible.
plotnine 0.5.1 has requirement pandas>=0.23.4, but you'll have pandas 0.22.0 which is incompatible.

 Clonning FastAI repository locally ...
Cloning into 'fastai_ml'...
remote: Enumerating objects: 23411, done.
remote: Total 23411 (delta 0), reused 0 (delta 0), pack-reused 23411
Receiving objects: 100% (23411/23411), 374.71 MiB | 31.10 MiB/s, done.
Resolving deltas: 100% (16495/16495), done.
Checking out files: 100% (758/758), done.


In [0]:
print ("\n Installing required libraries...")
!pip install --upgrade setuptools > /dev/null
!pip install feather > /dev/null
!pip install scikit-misc==0.1.0 > /dev/null
!pip install pdpbox==0.2.0 > /dev/null
!pip install treeinterpreter==0.2.2 > /dev/null
print ("\n Downloading datasets...")
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-train.7z
!7z x -y grocery-sales-train.7z > /dev/null
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-test.tar.gz
!tar xvf grocery-sales-test.tar.gz > /dev/null
print ("\n Importing libraries")
import pandas as pd
import os
import numpy as np


 Installing required libraries...
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-752iq1m6/feather/

--2019-02-14 17:36:40--  https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-train.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23818196 (23M) [application/octet-stream]
Saving to: ‘grocery-sales-train.7z’

grocery-sales-train 100%[===================>]  22.71M   110MB/s    in 0.2s    

2019-02-14 17:36:41 (110 MB/s) - ‘grocery-sales-train.7z’ saved [23818196/23818196]

--2019-02-14 17:36:45--  https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-test.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133,

# 1 Imports 

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [0]:
PATH = 'grocery-sales/'
!ls {PATH}

test.csv  train_basic.csv


# 2 Read data

In [0]:
types = {
    'id': 'int64',
    'item_nbr': 'int32',
    'store_nbr': 'int8',
    'unit_sales': 'float32',
    'onpromotion': 'object'
}

In [0]:
%%time
df_all = pd.read_csv(f'{PATH}train_basic.csv', parse_dates=['date'], dtype=types, 
                     infer_datetime_format=True)#, skiprows=range(1,100000000))

CPU times: user 2.58 s, sys: 140 ms, total: 2.72 s
Wall time: 2.74 s


In [0]:
df_all.onpromotion.fillna(False, inplace=True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

os.makedirs('tmp', exist_ok=True)
%time df_all.to_feather('tmp/raw_groceries')

CPU times: user 71.1 ms, sys: 67.7 ms, total: 139 ms
Wall time: 414 ms


In [0]:
%time df_all.describe(include='all')

CPU times: user 832 ms, sys: 486 µs, total: 833 ms
Wall time: 857 ms


,Unnamed: 0,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.997040e+06,1.997040e+06,1997040,1.997040e+06,1.997040e+06,1.997040e+06,1997040
unique,NaN,NaN,1684,NaN,NaN,NaN,2
top,NaN,NaN,2017-05-07 00:00:00,NaN,NaN,NaN,False
freq,NaN,NaN,1908,NaN,NaN,NaN,1528448
first,NaN,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,2.449852e+07,6.271723e+07,NaN,2.745692e+01,9.728621e+05,8.551911e+00,NaN
std,5.764959e+05,3.619772e+07,NaN,1.633284e+01,5.202940e+05,2.057796e+01,NaN
min,2.350000e+07,2.060000e+02,NaN,1.000000e+00,9.699500e+04,-1.768000e+03,NaN
25%,2.399926e+07,3.138601e+07,NaN,1.200000e+01,5.227210e+05,2.000000e+00,NaN


In [0]:
df_test = pd.read_csv(f'{PATH}test.csv',parse_dates=['date'], dtype=types, infer_datetime_format=True)

df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_test.onpromotion = df_all.onpromotion.astype(bool)
df_test.describe(include='all')

,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,1997040
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-27 00:00:00,NaN,NaN,False
freq,NaN,210654,NaN,NaN,1528448
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


In [0]:
df_all.tail()

,Unnamed: 0,id,date,store_nbr,item_nbr,unit_sales,onpromotion
1997035,25497035,73734366,2016-03-18,54,1464238,22.0,False
1997036,25497036,101679247,2016-12-31,49,1489899,29.0,True
1997037,25497037,124420242,2017-08-05,38,849080,2.0,False
1997038,25497038,35980557,2014-11-25,31,1463798,3.0,False
1997039,25497039,85369289,2016-07-18,28,362035,14.0,False


In [0]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [0]:
%time add_datepart(df_all, 'date')

CPU times: user 1.74 s, sys: 255 ms, total: 2 s
Wall time: 2 s


In [0]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

n_valid = len(df_test)
n_trn = len(df_all) - n_valid
train, valid = split_vals(df_all, n_trn)
train.shape, valid.shape

((623616, 19), (1373424, 19))

In [0]:
%%time
trn, y, nas = proc_df(train, 'unit_sales')
val, y_val, nas = proc_df(valid, 'unit_sales')

CPU times: user 1.04 s, sys: 232 ms, total: 1.27 s
Wall time: 1.27 s


# 3 Models

In [0]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val),
          m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [0]:
set_rf_samples(1000000)

In [0]:
%time x = np.array(trn, dtype=np.float32)

CPU times: user 586 ms, sys: 75.9 ms, total: 662 ms
Wall time: 663 ms


In [0]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=100, n_jobs=-1)
%time m.fit(x, y)

CPU times: user 3min 8s, sys: 47.6 ms, total: 3min 8s
Wall time: 1min 34s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=100, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [0]:
print_score(m)

[0.7751660488750103, 0.7933491325274509, 0.22885525419923247, 0.18939474725506789]


In [0]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=10, n_jobs=-1)
%time m.fit(x, y)
print_score(m)

CPU times: user 4min 10s, sys: 76.7 ms, total: 4min 10s
Wall time: 2min 5s
[0.5955681632494816, 0.7689149145027334, 0.5447925450732758, 0.23855720592124596]


In [0]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, n_jobs=2)
%time m.fit(x, y)
print_score(m)

CPU times: user 4min 37s, sys: 157 ms, total: 4min 37s
Wall time: 2min 19s
[0.3672129902421545, 0.7925169046500011, 0.8269456621416778, 0.19109451455114923]
